<a href="https://colab.research.google.com/github/elinoyam/FindMyParking/blob/WorkingWithColab/CalculateProbabilitiesFromGeojson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import numpy as np

In [ ]:
def find_osm_node_id(coordinate):
  lat = coordinate[0]
  lon = coordinate[1]
  # get the way id - request json format of the reverse method of OSM librery nominatim
  request_url = "https://nominatim.openstreetmap.org/reverse?format=geojson&osm_type=N&lat=" + f'{lat:f}' + "&lon=" + f'{lon:f}'
  #print(request_url)
  response = requests.get(request_url)
  way_id_response = json.loads(response.text)
  #print(way_id_response)
  osm_type = way_id_response['features'][0]['properties']['osm_type']
  node_id = way_id_response['features'][0]['properties']['osm_id'] 
  if osm_type != "node":
    print("not node:")
    print(node_id)

  return node_id

In [ ]:
def calculate_coords(list_of_old_coorinates):
  transformed_coordinates = []
  for old_cord in list_of_old_coorinates:
    old_lat = old_cord[0]
    old_lon = old_cord[1]
    # get the latitude and longitude in the right coordinates 
    # from the original format EPSG:25833 to format: EPSG:4326
    request_url = "https://epsg.io/trans?s_srs=25833&t_srs=4326&x="+str(old_lat)+"&y=" + str(old_lon)
    response = requests.get(request_url)
    transform_coordinates_response = json.loads(response.text)
    trans_lon = transform_coordinates_response['x']
    trans_lat = transform_coordinates_response['y']
    transformed_coordinates.append([float(trans_lat), float(trans_lon)])
  
  return transformed_coordinates


In [ ]:
def transform_line(line):
  old_coords = line['geometry']['coordinates'][0][0]
  transformed_coords = calculate_coords(old_coords)
  osm_id = find_osm_node_id(transformed_coords[0])
  # calculate parking area
  all_lats = [x for x,y in transformed_coords]
  all_lons = [y for x,y in transformed_coords]
  parking_area = polygon_area(all_lats, all_lons) # we multiply by 1000 to convet to m2 instead of km2
  # calculate number of parking spots in the parking area
  single_parking_area = 4.88 * 2.44
  number_of_parking = parking_area / single_parking_area
  print(number_of_parking)
  # get the label (parking empty/occupied)
  line_label = line['label_x']

  return osm_id, line_label, number_of_parking

In [ ]:
file_path = "parking.geojson"
# count the parking spots members:
empty_pk_spots = {}
occupied_pk_spots = {}

# open the file using geojson (less updated)
with open(file_path) as f:
    data_file = geojson.load(f)
    features_array = data_file['features']

    for feature in features_array:
      node_id, label, amount = transform_line(feature)
      if label == 'PK-space-occupied':
        occupied_count = occupied_pk_spots.setdefault(way_id, 0)
        occupied_pk_spots[way_id] = occupied_count + 1
      elif label == 'PK-space-empty':
        empty_count = empty_pk_spots.setdefault(way_id, 0)
        empty_pk_spots[way_id] = empty_count + 1 


In [ ]:
line = '{ "type": "Feature", "properties": { "id": 1, "label_x": "PK-space-occupied", "True_or_False": null, "label_y": null, "nearest_geometry_dis": null, "parking_area_m2": null, "layer": "Annotaions Predictions", "path": "F:/Berlin-DOP_2020/Predictions/Previously created maps/Annotations/Annotaions.geojson" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ 391008.0878, 5821088.3568 ], [ 391002.407, 5821095.493 ], [ 391004.3319, 5821096.9484 ], [ 391009.8718, 5821089.4366 ], [ 391008.1817, 5821088.2629 ], [ 391008.0878, 5821088.3568 ] ] ] ] } }'
#line = line.replace("null", "None")
json_line = json.loads(line)
transform_line(json_line)

1.7470470147182449


In [ ]:
osm_id = find_osm_node_id([52.54203249, 13.41329482])
print(osm_id)

692194370


In [ ]:
input_coord = [ [ 392395.9, 5822510.25 ], [ 392396.65, 5822510.8 ], [ 392398.25, 5822513.5 ], [ 392398.15, 5822513.8 ], [ 392397.6, 5822514.35 ], [ 392397.1, 5822514.65 ], [ 392396.6, 5822514.55 ], [ 392396.25, 5822514.2 ], [ 392395.15, 5822512.6 ], [ 392394.95, 5822512.1 ], [ 392394.95, 5822511.6 ], [ 392395.35, 5822510.7 ], [ 392395.9, 5822510.25 ]]

output_coord = calculate_coords(input_coord)

print(output_coord)

for cord in output_coord:
  found_id = find_osm_node_id(cord)
  print(found_id)

[[52.54203249, 13.41329482], [52.54203758, 13.41330569], [52.54206216, 13.4133284], [52.54206483, 13.41332683], [52.54206967, 13.41331855], [52.54207226, 13.41331108], [52.54207127, 13.41330374], [52.54206805, 13.4132987], [52.54205346, 13.413283], [52.54204892, 13.41328021], [52.54204443, 13.41328038], [52.54203642, 13.41328656], [52.54203249, 13.41329482]]
https://nominatim.openstreetmap.org/reverse?format=geojson&osm_type=N&lat=52.542032&lon=13.413295
{'type': 'FeatureCollection', 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'features': [{'type': 'Feature', 'properties': {'place_id': 5741901, 'osm_type': 'node', 'osm_id': 692194370, 'place_rank': 30, 'category': 'place', 'type': 'house', 'importance': 0, 'addresstype': 'place', 'name': None, 'display_name': '86, Pappelallee, Helmholtzkiez, Prenzlauer Berg, Pankow, Berlin, 10437, Deutschland', 'address': {'house_number': '86', 'road': 'Pappelallee', 'neighbourhood': 'Helmholtzkiez', 'suburb': '

In [ ]:
# first way to calculate area on the earth surface

!pip install area
from area import area
poly_obj = {'type':'Polygon','coordinates':[[[52.54203249, 13.41329482], [52.54203758, 13.41330569], [52.54206216, 13.4133284], [52.54206483, 13.41332683], [52.54206967, 13.41331855], [52.54207226, 13.41331108], [52.54207127, 13.41330374], [52.54206805, 13.4132987], [52.54205346, 13.413283], [52.54204892, 13.41328021], [52.54204443, 13.41328038], [52.54203642, 13.41328656], [52.54203249, 13.41329482]]]}
area(poly_obj)

In [ ]:
# second (and more precise) way for calculating the area on earth surface

def polygon_area(lats, lons, radius = 6378137):
    """
    Computes area of spherical polygon, assuming spherical Earth. 
    Returns result in ratio of the sphere's area if the radius is specified.
    Otherwise, in the units of provided radius.
    lats and lons are in degrees.
    """
    from numpy import arctan2, cos, sin, sqrt, pi, power, append, diff, deg2rad
    lats = np.deg2rad(lats)
    lons = np.deg2rad(lons)

    # Line integral based on Green's Theorem, assumes spherical Earth

    #close polygon
    if lats[0]!=lats[-1]:
        lats = append(lats, lats[0])
        lons = append(lons, lons[0])

    #colatitudes relative to (0,0)
    a = sin(lats/2)**2 + cos(lats)* sin(lons/2)**2
    colat = 2*arctan2( sqrt(a), sqrt(1-a) )

    #azimuths relative to (0,0)
    az = arctan2(cos(lats) * sin(lons), sin(lats)) % (2*pi)

    # Calculate diffs
    # daz = diff(az) % (2*pi)
    daz = diff(az)
    daz = (daz + pi) % (2 * pi) - pi

    deltas=diff(colat)/2
    colat=colat[0:-1]+deltas

    # Perform integral
    integrands = (1-cos(colat)) * daz

    # Integrate 
    area = abs(sum(integrands))/(4*pi)

    area = min(area,1-area)
    if radius is not None: #return in units of radius
        return area * 4*pi*radius**2
    else: #return in ratio of sphere total area
        return area